## Импорт и определение библиотек

In [3]:
# ставим разные пакеты оттуда
!pip install langchain langchain_experimental langchain-openai openai tiktoken huggingface_hub langchain-huggingface -q
!pip install beautifulsoup4 pypdf sentence-transformers faiss-cpu langchainhub -q

In [4]:
# импортируем библиотеки
import os
import requests
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool

## Импортируем библиотеки для RAG
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
#from utils import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import initialize_agent, AgentType

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# для использования ключа из LLM_bot
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py -q

# Готовим коктейль с помощью RAG
## Немного `Document Loader`

In [ ]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))

In [1]:
!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()

100%|██████████| 2/2 [00:17<00:00,  8.65s/it]


## Исполним `Text splitters` и разобьём на чанки

In [5]:
splitter = RecursiveCharacterTextSplitter(
        '\n', 
        chunk_size=1000, 
        chunk_overlap=100
    )

split_documents = []
for page in data:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

len(split_documents)

1193

In [6]:
split_documents[1190].page_content

'Во внутреннем оформлении использованы фотографии и иллюстрации: 3523studio, Africa Studio, Aleksey Akimov, AlexLMX, Anna Nass, apolonia, BMCL, Bochkarev Photography, Brent Hofacker, Chris Bence, DavideAngelini, dibrova, Dietmar Rauscher, Dmitrii Rud, Dmity Trush, EHStockphoto, Ekat- erina Bratova, Elena Gordeichik, Elena Prozhivina, Evgeny Karandaev, Foodio, Goskova Tatiana, grafvision, gresei, Hortimages, hvoya, Igor Normann, Ivan Mateev, Jacek Fu- lawka, JethroT, Kasia, Katarzyna Hurova, Kravtzov, krolya25, lhmfoto, Liliya Kandrashevich, M. Unal Ozmen, mama_mia, Mateusz Gzik, Micaela Fiorellini, monticello, Olga Moonlight, Palmer Kane LLC, Paolo Bernardotti Studio, Pixel-Shot, Ponomareva Mariia, RubelHossainCreative, Rudolfovich, smspsy, TheLionRoar, VanoVasaio, Velimir Zeland, viennetta, VikaGeyder, wiratho, Y production / Shutterstock.com Используется по лицензии от Shutterstock.com'

## Чутка `embeddings с HuggingFace`

In [7]:
# ai-sage/Giga-Embeddings-instruct
!pip install sentence-transformers -q

In [8]:
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="DeepPavlov/rubert-base-cased",#"cointegrated/LaBSE-en-ru", 
    model_kwargs={
        "device": "cpu"
                  } 
    # model_name="sentence-transformers/all-MiniLM-L6-v2", 
)

/tmp/ipykernel_295344/2883937955.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(
No sentence-transformers model found with name DeepPavlov/rubert-base-cased. Creating a new one with mean pooling.
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_rel

## Наполним чашу `Vector Store`

In [9]:
"""# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(split_documents, hf_embeddings_model)
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db") """

# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

## Можно полирнуть `Retriver`, но не нужно

In [10]:
# Теперь можно выполнять поиск по загруженной базе
#results = db_embed.similarity_search("состав коктейля зелёная фея")

# Обработка результатов
#for result in results:
#    print(result)

# db_embed.similarity_search("состав коктейля зелёная фея")

## Дополним `Retriever` для результата

# вытаскиваем данные с помощью ретривера
# Самый частый кейс - использование векторного хранилища и его методов для получения документов
retriever = db_embed.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    core_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)
retriever.get_relevant_documents(
    "состав коктейля зелёная фея"
)[0]

/tmp/ipykernel_295344/2905463311.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(


Document(id='c02d9019-a7c1-4be1-b3ec-a0f90a68a426', metadata={'source': '../docs/Bortnik_1000.pdf'}, page_content='«Absolut Citron» изготавливается на основе технологии, позволяющей сохранять тончайшие ароматы цитрусовых при идеальной прозрачности напитка. На рынке появи- лась в 1988 г. Была представлена как освежа ющий напи- ток, прекрасно гармонирующий с современным стилем жизни. Ароматизаторами служат натуральные цитрусовые эссенции. Водка яркого и чистого вкуса, с характерной сухостью. Доминируют тона лимона. Грейпфрут и манда- рин придают вкусу полноту, а лайм — свежесть. «Absolut Citron» можно подавать в чистом виде или со льдом, а также использовать в составе множества коктейлей. Например, она является идеальным ингредиентом для коктейля «Водка Мар тини».\n\n«Absolut Kurant» — самый «молодой» член семейства «Абсолютов» в Швеции. Издавна здесь делали лечебные спиртовые настойки на черной смородине (по-шведски «kurant»). Уникальное сочетание ягодной свежести и\n\n140\n\nПРИЛОЖЕНИЕ

# Миксуем рецепт функциями и агентом

In [11]:
def get_cocktail_recipes(ingredients_str, iba_category=None):
    # Системный промпт для бармена
    system_prompt = "Ты опытный бармен, специализирующийся на алкогольных коктейлях."
    
    # Задаем промт пользователя с учётом категории
    if iba_category:
        user_prompt = (f"Напиши, какие коктейли из категории '{iba_category}' можно изготовить из представленных ниже ингредиентов. Учитывай только коктейли, в состав которых входят только все перечисленные ингредиенты. "
                       f"Описание не должно содержать обобщающих слов и фраз. "
                       f"Для каждого из представленных тобой коктейлей после перечиления ингредиентов опиши подробный способ приготовления. "
                       f"Также укажи, как ещё составляющие входят в коктейли и в какой пропорции изготавливаются. "
                       f"Дополнительно укажи, какие ингредиенты нужно добавить, чтобы получились другие коктейли. "
                       f"\nИнгредиенты: {ingredients_str}")
    else:
        user_prompt = (f"Напиши, какие коктейли из всех категорий IBA можно изготовить из представленных ниже ингредиентов. Учитывай только коктейли, в состав которых входят только все перечисленные ингредиенты. "
                       f"Описание не должно содержать обобщающих слов и фраз. "
                       f"Для каждого из представленных тобой коктейлей после перечиления ингредиентов опиши подробный способ приготовления. "
                       f"Также укажи, как ещё составляющие входят в коктейли и в какой пропорции изготавливаются. "
                       f"Дополнительно укажи, какие ингредиенты нужно добавить, чтобы получились другие коктейли. "
                       f"\nИнгредиенты: {ingredients_str}")

    # Формируем список сообщений для модели
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Получаем ответ от модели
    response = llm(messages).content
    
    # Возвращаем полученный ответ
    return response

"""# Создаем инструмент для поиска коктейлей
def cocktail_search_tool(ingredients_str, iba_category):
    return get_cocktail_recipes(ingredients_str, iba_category)

cocktail_search_tool = create_retriever_tool(
    cocktail_search_tool,
    "cocktail_search",
    "Searches for cocktail recipes based on ingredients and category"
)"""

# Создаем инструмент для поиска коктейлей
cocktail_search_tool = Tool(
    name="cocktail_search",
    func=get_cocktail_recipes,
    description="Searches for cocktail recipes based on ingredients and category"
)

## Задаём общие параметры

In [12]:
# Запрашиваем у пользователя ввод ингредиентов
user_input = input("Введите ингредиенты, разделенные запятыми: ")

# Преобразуем введенную строку в список, удаляя лишние пробелы
ingredients = [ingredient.strip() for ingredient in user_input.split(",")]

# Форматируем строку с использованием join для соединения элементов массива
ingredients_str = ", ".join(ingredients)

# Список категорий IBA
iba_categories = [
    "незабываемые", 
    "новая классика", 
    "напитки новой эры", 
    "не вошедшие"
                  ]

# Просим пользователя выбрать категорию или оставить пустым для поиска по всем
#    """Выберите категорию IBA (введите число):
#        1 - Незабываемые
#        2 - Новая классика
#        3 - Напитки новой эры
#        4 - Не вошедшие"""
category_input = input("Выберите категорию IBA (или оставьте пустым для всех категорий): ")

# Приводим выбор к категории
try:
    iba_category = iba_categories[int(category_input) - 1] if category_input.isdigit() and 1 <= int(category_input) <= 4 else None
except ValueError:
    iba_category = None  # Если ничего не выбрано, ищем по всем категориям

## Инициализация `llm` и наслаждение от `agent`

In [13]:
# Если используете ключ из курса по LLM, запустите эту ячейку
from utils import ChatOpenAI

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break
#print(course_api_key)

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

agent = initialize_agent(
    tools=[cocktail_search_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/tmp/ipykernel_295344/1777797833.py:16: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [32]:
"""## Если не хотите платить денежков, то запустите эту ячейку
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('HUGGINGFACEHUB_API_TOKEN'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# Создаем объект языкового модели
llm = HuggingFaceEndpoint(
      #repo_id="IlyaGusev/saiga_llama3_8b",
      repo_id="mistralai/Mistral-7B-Instruct-v0.3",
      task="text-generation",  # Вид задачи, в нашем случае - генерация текста
      huggingfacehub_api_token = course_api_key
    )"""

In [77]:
"""messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]"""
"""# Получаем ответ от модели
response = llm(messages).content

# Выводим ответ
print("Рецепт коктейлей:\n")
print(response)"""

# Получаем коктейль

In [14]:
# Используем db_embed для поиска рецептов на основе ингредиентов
def search_cocktails_in_db(ingredients_str):
    # Здесь вы можете реализовать поиск по вашей базе данных FAISS
    # Например, используя векторизацию ингредиентов и выполнение поиска
        # Предположим, что у вас есть функция для этого:
    results = db_embed.similarity_search(ingredients_str)  # Псевдокод для поиска
    return results

# Выполняем поиск в базе данных
search_results = search_cocktails_in_db(ingredients_str)

# Обрабатываем результаты поиска и формируем запрос к агенту
if search_results:
    # Извлекаем нужные данные из результатов поиска (например, текст документа)
    cocktail_names = [result.page_content for result in search_results]  # Используем page_content для получения текста
    cocktail_names_str = ", ".join(cocktail_names)
    
    # Вызываем функцию получения рецептов с найденными коктейлями
    response = get_cocktail_recipes(cocktail_names_str, iba_category)
else:
    response = "Коктейли не найдены по заданным ингредиентам."

# Выводим ответ
print("Рецепт коктейлей:\n")
print(response)

/tmp/ipykernel_295344/1415629978.py:28: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content


Рецепт коктейлей:

Коктейль "Аквавит Мартини":
Ингредиенты: Аквавит.
Способ приготовления: Налейте 60 мл аквавита в стеклянную смесительную чашу с льдом. Перемешайте и процедите в охлажденный коктейльный бокал. Гарнируйте оливкой или ломтиком лимона.

Для приготовления коктейля "Аквавит Мартини" добавьте водку и вермут.

Коктейль "Аквавит Тоник":
Ингредиенты: Аквавит, тоник.
Способ приготовления: Наполните высокий стакан льдом, добавьте 50 мл аквавита и долейте тоник. Перемешайте и украсьте долькой лимона или ломтиком огурца.

Для приготовления коктейля "Аквавит Тоник" добавьте лимонный сок и листья мяты.

Коктейль "Аквавит Скрю":
Ингредиенты: Аквавит, лимонный сок, сахарный сироп, яичный белок.
Способ приготовления: В шейкере с льдом смешайте 50 мл аквавита, 20 мл лимонного сока, 10 мл сахарного сиропа и яичный белок. Хорошо взбейте и процедите в охлажденный коктейльный бокал.

Для приготовления коктейля "Аквавит Скрю" добавьте ангостуру биттерс и ломтик апельсина.

Для приготовления 

## Если используем модель с Hugging Face

In [ ]:
"""# Если выбрали модель с HuggingFace
# Объединяем системный и пользовательский промты в одну строку
full_prompt = f"{system_prompt}\n{user_prompt}"

# Получаем ответ от модели
response = llm(full_prompt)

# Выводим ответ
print(response)"""

## Меняем стиль ответа

In [ ]:
# Функция выбора стиля ответа

def choose_style():
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "Стиль барного меню"  # Стиль по умолчанию
    }
    #print("Доступные стили ответа:")
    for key, value in styles.items():
        print(f"{key}: {value}")
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "style4")

style = choose_style()
prompt = PromptTemplate(input_variables=['output_text', 'style'],
                        template='''Перепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:''')
style_changer_chain = prompt | llm

styled_response = style_changer_chain.invoke({'output_text': response, 'style': style}).content
print("\nОтвет в выбранном стиле:")
print(styled_response)

# Разные способы поиска цен на ингредиенты

## Поиск с помощью агента
1. Инструменты LangChain:
    - Мы можем использовать инструменты, такие как `DuckDuckGoSearchRun` (для поиска через DuckDuckGo), или создать собственные инструменты для взаимодействия с API магазинов (например, Winelab, SimpleWine и других).
2. Агент LangChain:
    - Агент будет обрабатывать запросы, отправлять их в поисковые инструменты и агрегировать ответы.
3. Поиск стоимости товара:
    - Агент использует инструменты, чтобы найти релевантную информацию о стоимости алкогольных ингредиентов.

In [ ]:
pip install -U duckduckgo-search -q

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI

# Подключение LLM
#llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)

# Создаем поисковый инструмент DuckDuckGo
search = DuckDuckGoSearchRun()

# Добавляем инструмент в список
tools = [
    Tool(
        name="Search ingredient",
        func=search.run,
        description="Используйте этот инструмент для поиска информации об ингредиентах и ценах на самые популярные из них."
    )
]

# Инициализация агента
price_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",  # Используем Zero-shot агент
    verbose=True
)

# Формируем промт для поиска цен на товар
ingredient = "бурбон"
city = "Санкт-Петербург"
prompt = (
    f"Найдите цену на продукт '{ingredient}' в интернет-магазинах {city}. "
    """Если вариантов несколько, укажите 3 лучших результата. 
    Если в ответе название пишется на языке производителя, то оставь как есть. Название валюты и объём должны быть указаны на русском языке. 
    Укажи ответ в формате: название экземпляра, стоимость в среднем ценовом диапазоне, объём, ссылка на магазин, если имеется; если нет, то ничего не писать. 
    Если вариантов ответа несколько, то отобрази их в виде вертикального списка, где каждый новый вариант отображается с новой строки. """
)

# Выполнение агента
result = price_agent.run(prompt)

print(result)


## Поиск с помощью `RAG`

Общая стратегия:

1. Использование `WebLoader` для сбора данных:
   - скачиваем страницы для каждой категории алкоголя с сайтов, таких как `Winelab`.
   - сохраняем их текстовое содержимое (например, названия и цены продуктов) в векторное хранилище.
1. Создание векторного хранилища:
    - разбиваем загруженные данные на чанки.
    - индексируем их с помощью векторизатора (например, FAISS или Chroma).
1. Поиск ингредиентов:
    - при запросе пользователя (например, "водка"), извлекаем соответствующую информацию из хранилища.
1. Использование `LangChain`:
    - для загрузки данных используем WebBaseLoader из LangChain.
    - для индексации данных — инструменты LangChain, такие как Chroma или FAISS.
    - для обработки запросов пользователя — цепочки, основанные на LLM (например, `OpenAI API`).

In [ ]:
pip install bs4 langchain-openai faiss-cpu -q

In [ ]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Установите ваш ключ API
# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# 1. Сбор данных с категорий сайта
def fetch_and_index_alcohol_data():
    categories = {
        "absinthe": "https://www.winelab.ru/catalog/krepkiy-alkogol-absent",
        "vodka": "https://www.winelab.ru/catalog/krepkiy-alkogol-vodka",
        "whiskey": "https://www.winelab.ru/catalog/krepkiy-alkogol-viski",
    }

    documents = []
    for category, url in categories.items():
        try:
            loader = WebBaseLoader(url)
            category_docs = loader.load()
            documents.extend(category_docs)
            print(f"Загружено {len(category_docs)} документов из {url}.")
        except Exception as e:
            print(f"Ошибка при загрузке данных из {url}: {e}")

    print(f"Всего загружено документов: {len(documents)}")
    return documents

# 2. Разбивка текста на чанки
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    print(f"Всего создано чанков: {len(chunks)}")
    return chunks


# 3. Индексация данных с использованием FAISS
def create_faiss_index(chunks):
    if not chunks:
        raise ValueError("Нет данных для индексации. Проверьте загрузку и разбивку документов.")
    
    embeddings = OpenAIEmbeddings(openai_api_key=course_api_key)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore


# 4. Создание цепочки RAG для поиска
def create_retrieval_chain(vectorstore):
    retriever = vectorstore.as_retriever()
    llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)
    chain = RetrievalQA(llm=llm, retriever=retriever)
    return chain

# Основная функция
def main():
    print("Сбор данных...")
    documents = fetch_and_index_alcohol_data()
    print("Разделение на чанки...")
    chunks = split_documents(documents)
    print("Создание векторного индекса...")
    vectorstore = create_faiss_index(chunks)
    print("Создание цепочки RAG...")
    rag_chain = create_retrieval_chain(vectorstore)

    # Поиск цены ингредиента
    ingredient_query = "водка"
    print(f"Поиск для: {ingredient_query}")
    result = rag_chain.run(ingredient_query)
    print(f"Результат: {result}")

if __name__ == "__main__":
    main()

## Поиск `в лоб`

In [ ]:
import requests
import re

def search_price_with_specific_cookies(ingredient):
    """
    Функция для поиска цен на алкогольные ингредиенты с индивидуальными куки для каждого магазина.
    """
    stores = [
        {
            "name": "Winelab",
            "url": lambda ing: f"https://www.winelab.ru/search/facets?text={ing}",
            "cookies": {"age-confirmed": "1"},
            "title_pattern": r'<a class="product-card__link" .*? title="(.*?)"',
            "price_pattern": r'<div class="price__main-value">(.*?)</div>',
            "link_pattern": r'<a class="product-card__link" href="(.*?)"',
            "link_prefix": "https://www.winelab.ru"
        },
        {
            "name": "SimpleWine",
            "url": lambda ing: f"https://simplewine.ru/catalog/?q={ing}",
            "cookies": {"age-confirm": "1"},
            "title_pattern": r'<a class="product-card__title-link" .*?>(.*?)</a>',
            "price_pattern": r'<div class="product-card__price-current">(.*?)</div>',
            "link_pattern": r'<a class="product-card__title-link" href="(.*?)"',
            "link_prefix": "https://simplewine.ru"
        }
    ]

    results = []

    for store in stores:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"
            }
            response = requests.get(store["url"](ingredient), headers=headers, cookies=store["cookies"], timeout=5)
            if response.status_code == 200:
                # Извлечение данных о товарах
                titles = re.findall(store["title_pattern"], response.text)
                prices = re.findall(store["price_pattern"], response.text)
                links = re.findall(store["link_pattern"], response.text)

                for title, price, link in zip(titles[:3], prices[:3], links[:3]):
                    full_link = f"{store['link_prefix']}{link}"
                    results.append(f"- {title.strip()} {price.strip()} рублей {full_link} ({store['name']})")

                if results:
                    break  # Если найдены результаты, завершаем поиск
            else:
                results.append(f"Ошибка доступа к {store['name']} для {ingredient}: статус {response.status_code}")
        except Exception as e:
            results.append(f"Ошибка при выполнении поиска в {store['name']} для {ingredient}: {str(e)}")

    if not results:
        results.append(f"Данные о наличии в продаже {ingredient} не найдены.")

    return "\n".join(results)

# Пример использования
ingredient_name = "джин"
print(search_price_with_specific_cookies(ingredient_name))

In [ ]:
# Разбор коктейля и ингредиентов для поиска цен
pattern = r"\*\*(.*?)\*\*:.*?((?:\n\s*-\s*.*?\n)+)"
match = re.search(pattern, response, re.DOTALL)
if match:
    cocktail_name = match.group(1)
    ingredients_section = match.group(2)
    print(f"\nКоктейль: {cocktail_name}")
    print(f"Ингредиенты:")
    print(ingredients_section)
    
    # Поиск цен для каждого ингредиента
    ingredient_lines = ingredients_section.strip().split("\n")
    for line in ingredient_lines:
        ingredient_match = re.search(r"-\s*\d+\s*мл\s*([^\n]+)", line)
        if ingredient_match:
            ingredient_name = ingredient_match.group(1)
            print(f"\nИщем цену для: {ingredient_name}")
            prices = search_price(ingredient_name)
            print(prices)
        else:
            print(f"\nНе удалось извлечь название ингредиента из строки: {line}")
else:
    print("Не удалось найти информацию о коктейле и ингредиентах в ответе модели.")

## Поиск с помощью Selenium

In [ ]:
pip install selenium -q

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def search_price_with_selenium(ingredient):
    """
    Поиск цен на ингредиенты с использованием Selenium.
    """
    # Укажите путь к вашему WebDriver
    service = Service('/home/elijah/Documents/chromedriver-linux64/chromedriver')  # Укажите корректный путь к ChromeDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Запуск в безголовом режиме
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    try:
        # Магазин Winelаб
        driver.get("https://www.winelab.ru")

        # Подтверждение возраста
        age_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "age-confirm"))
        )
        age_button.click()

        # Поиск ингредиента
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "text"))  # Обратите внимание на имя поля
        )

        # Добавление небольшой задержки для обеспечения доступности элемента
        time.sleep(1)

        # Используем JavaScript для установки значения в поле ввода
        driver.execute_script("arguments[0].value = arguments[1];", search_box, ingredient)
        
        # Теперь вызываем событие input для имитации ввода текста
        search_box.send_keys(Keys.RETURN)

        # Найдите кнопку отправки и нажмите ее (если это необходимо)
        # search_button = driver.find_element(By.CLASS_NAME, "js_search_button")
        # search_button.click()

        # Сбор результатов
        products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card"))
        )
        
        for product in products[:3]:  # Берём максимум 3 результата
            title = product.find_element(By.CLASS_NAME, "product-card__title").text
            price = product.find_element(By.CLASS_NAME, "price__main-value").text
            
            results.append(f"- {title} {price.strip()} рублей")

    except Exception as e:
        results.append(f"Ошибка при обработке ингредиента {ingredient}: {str(e)}")
    finally:
        driver.quit()

    return results if results else f"Данные о наличии в продаже {ingredient} не найдены."

# Пример использования
ingredient_name = "водка"
print(search_price_with_selenium(ingredient_name))